In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.utils import image_dataset_from_directory

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

2.16.2
Num GPUs Available:  1


Data Preprocessing 

In [2]:
training_data = "data/Training"
testing_data = "data/Testing"

class_names = ['glioma', 'meningioma', 'no_tumor', 'pituitary']

def normalize_image(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_generator = image_dataset_from_directory(
    training_data,
    subset='training',
    seed=50,
    image_size=(256, 256),
    batch_size=64,
    validation_split=0.1,
).map(normalize_image)

validation_generator = image_dataset_from_directory(
    training_data,
    subset='validation',
    seed=50,
    image_size=(256, 256),
    batch_size=64,
    validation_split=0.3,
).map(normalize_image)

test_generator = image_dataset_from_directory(
    testing_data,
    image_size=(256, 256),
    batch_size=64,
    seed=50
).map(normalize_image)


Found 5712 files belonging to 4 classes.
Using 5141 files for training.
Found 5712 files belonging to 4 classes.
Using 1713 files for validation.
Found 1311 files belonging to 4 classes.


Show graph

In [3]:

# Code for plotting
# def show(images, labels):
#     plt.figure(figsize=(10, 10))
#     for i in range(16):
#         plt.subplot(4, 4, i + 1)
#         plt.xticks([])
#         plt.yticks([])
#         plt.grid(False)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.xlabel(class_names[labels[i]])
#     plt.show()

# show(train_images, train_labels)

Model

In [4]:
from tensorflow.keras.optimizers import Adam
model = models.Sequential()

model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(4, activation='softmax'))
print(model.summary())


model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

batch_size = 64
epochs = 10

/opt/anaconda3/envs/tf310/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     4,194,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,256,100 (16.24 MB)

 Trainable params: 4,256,100 (16.24 MB)

 Non-trainable params: 0 (0.00 B)

None


Fit

In [5]:
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
)


Epoch 1/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 37s 422ms/step - accuracy: 0.5819 - loss: 0.9495 - val_accuracy: 0.8377 - val_loss: 0.4600
Epoch 2/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 38s 464ms/step - accuracy: 0.8202 - loss: 0.4785 - val_accuracy: 0.8938 - val_loss: 0.2984
Epoch 3/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 36s 442ms/step - accuracy: 0.8688 - loss: 0.3463 - val_accuracy: 0.9037 - val_loss: 0.2612
Epoch 4/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 36s 443ms/step - accuracy: 0.9056 - loss: 0.2576 - val_accuracy: 0.9405 - val_loss: 0.1868
Epoch 5/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 36s 438ms/step - accuracy: 0.9298 - loss: 0.1936 - val_accuracy: 0.9550 - val_loss: 0.1539
Epoch 6/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 235s 3s/step - accuracy: 0.9473 - loss: 0.1419 - val_accuracy: 0.9323 - val_loss: 0.1779
Epoch 7/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.9582 - loss: 0.1151 - val_accuracy: 0.9743 - val_loss: 0.1005
Epoch 8/10
81/81 ━━━━━━━━━━━━━━━━━━━━ 58s 714ms/step - accuracy: 0.9686 - loss: 0.0858 - val_accuracy

Evaulation

In [7]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy:.2f}')
print(f'Test loss: {test_loss:.2f}')

21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9516 - loss: 0.1529
Test accuracy: 0.96
Test loss: 0.18


Save models

In [ ]:
model.save("model.h5")
model.save("Neual_network_model")

Prediction